In [1]:
%cd '/content/drive/MyDrive/NLU_NCKH/notebook/'

/content/drive/MyDrive/NLU_NCKH/notebook


In [2]:
!pip install --q emoji underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 27.3 MB/s eta 0:00:00


# Import libraty

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
# from python_calamine.pandas import pandas_monkeypatch

In [ ]:
# import re
# import pandas as pd

# # Read the file line by line
# restaurant_train = './Res_ABSA/Train.txt'
# restaurant_test = './Res_ABSA/Test.txt'
# restaurant_dev = './Res_ABSA/Dev.txt'

# def make_dataframe(src):
#   with open(src, 'r', encoding='utf-8') as file:
#     lines = file.readlines()

#   input_str = ''.join(lines)

#   entries = input_str.split('\n\n')

#   entries = [entry for entry in entries if entry.strip()]

#   input_data = [entry.splitlines() for entry in entries]

#   return pd.DataFrame(input_data, columns=['stt', 'text', 'labels'])


# df_train_res = make_dataframe(restaurant_train)
# df_test_res = make_dataframe(restaurant_test)
# df_dev_res = make_dataframe(restaurant_dev)

# df_train_res.head()

# columns = set()
# for label in df_train_res['labels']:
#     sl = label.split(',')
#     sl = {s.replace('{', '').replace('}', '').replace(' ', '').replace('positive', '').replace('negative', '').replace('neutral', '') for s in sl}
#     sl.discard('')
#     columns.update(sl)

# columns = list(columns)
# len(columns), columns

# Load data and format data -> csv

In [34]:
aspects = [
 'FACILITIES#CLEANLINESS',
 'FACILITIES#COMFORT',
 'FACILITIES#DESIGN&FEATURES',
 'FACILITIES#GENERAL',
 'FACILITIES#MISCELLANEOUS',
 'FACILITIES#PRICES',
 'FACILITIES#QUALITY',
 'FOOD&DRINKS#MISCELLANEOUS',
 'FOOD&DRINKS#PRICES',
 'FOOD&DRINKS#QUALITY',
 'FOOD&DRINKS#STYLE&OPTIONS',
 'HOTEL#CLEANLINESS',
 'HOTEL#COMFORT',
 'HOTEL#DESIGN&FEATURES',
 'HOTEL#GENERAL',
 'HOTEL#MISCELLANEOUS',
 'HOTEL#PRICES',
 'HOTEL#QUALITY',
 'LOCATION#GENERAL',
 'ROOMS#CLEANLINESS',
 'ROOMS#COMFORT',
 'ROOMS#DESIGN&FEATURES',
 'ROOMS#GENERAL',
 'ROOMS#MISCELLANEOUS',
 'ROOMS#PRICES',
 'ROOMS#QUALITY',
 'ROOM_AMENITIES#CLEANLINESS',
 'ROOM_AMENITIES#COMFORT',
 'ROOM_AMENITIES#DESIGN&FEATURES',
 'ROOM_AMENITIES#GENERAL',
 'ROOM_AMENITIES#MISCELLANEOUS',
 'ROOM_AMENITIES#PRICES',
 'ROOM_AMENITIES#QUALITY',
 'SERVICE#GENERAL'
]

In [35]:
len(set(aspects))

34

In [36]:
restaurant_train = '/content/drive/MyDrive/NLU_NCKH/notebook/hotel_ABSA/Train.txt'
restaurant_test = './hotel_ABSA/Test.txt'
restaurant_dev = './hotel_ABSA/Dev.txt'

def label_encoder(label, aspects):
    y = [0] * len(aspects)
    ap_stm = re.findall('{(.+?), (\w+)}', label)
    for aspect, sentiment in ap_stm:
        idx = aspects.index(aspect)
        y[idx] = sentiment

    return y

def init_data(df):
    X = df.drop(['review'], axis=1)
    y = df.drop(['review'], axis=1).replace({'negative': 1,
                    'neutral': 2,
                    'positive': 3}).astype(np.uint8)

    print('X.shape:', X.shape, 'y.shape:', y.shape)
    return X, y

def txt2df(filepath, aspect):
    with open(filepath, 'r', encoding='utf-8-sig') as txt:
        data = txt.read().split('\n')

    df = pd.DataFrame()
    df['review'] = [review for review in data[1::4]]
    df[aspect] = [label_encoder(label, aspect) for label in data[2::4]]
    df[aspect] = df[aspect].replace({'negative': 1,
                    'neutral': 2,
                    'positive': 3}).astype(np.uint8)
    # print(aspects)
    return df


train = txt2df(restaurant_train, aspects)
dev = txt2df(restaurant_dev, aspects)
test = txt2df(restaurant_test, aspects)

Xtrain, ytrain = init_data(train)
Xdev,   ydev   = init_data(dev)
Xtest,  ytest  = init_data(test)

X.shape: (7180, 34) y.shape: (7180, 34)
X.shape: (795, 34) y.shape: (795, 34)
X.shape: (2030, 34) y.shape: (2030, 34)


# Preprocessing

In [29]:
# https://github.com/nguyenvanhieuvn/text-classification-tutorial/blob/master/text_classification_tutorial.ipynb
# https://nguyenvanhieu.vn/phan-loai-van-ban-tieng-viet

import regex as re
import string
import emoji

#from vncorenlp import VnCoreNLP
import underthesea
from nltk import flatten


# Remove HTML code
def remove_HTML(text):
    return re.sub(r'<[^>]*>', '', text)

# Standardize unicode
def convert_unicode(text):
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    charutf8 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    char1252 = char1252.split('|')
    charutf8 = charutf8.split('|')

    dic = {}
    for i in range(len(char1252)): dic[char1252[i]] = charutf8[i]
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dic[x.group()], text
    )


# Standardize accent typing
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)


def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True


def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)


def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


# Normalize acronyms
# !wget https://gist.githubusercontent.com/nguyenvanhieuvn/7d9441c10b3c2739499fc5a4d9ea06fb/raw/df939245b3e841b62af115be4dcb3516dadc9fc5/teencode.txt
replace_list = {
    'ô kêi': 'ok', 'okie': 'ok', 'o kê': 'ok', 'okey': 'ok', 'ôkê': 'ok', 'oki': 'ok', 'oke': 'ok', 'okay': 'ok', 'okê': 'ok',
    'tks': 'cảm ơn', 'thks': 'cảm ơn', 'thanks': 'cảm ơn', 'ths': 'cảm ơn', 'thank': 'cảm ơn',
    'kg': 'không', 'not': 'không', 'k': 'không', 'kh': 'không', 'kô': 'không', 'hok': 'không', 'ko': 'không', 'khong': 'không', 'kp': 'không phải',
    'he he': 'tích cực', 'hehe': 'tích cực', 'hihi': 'tích cực', 'haha': 'tích cực', 'hjhj': 'tích cực', 'thick': 'tích cực',
    'cc': 'tiêu cực', 'huhu': 'tiêu cực', 'cute': 'dễ thương',

    'sz': 'cỡ', 'size': 'cỡ',
    'wa': 'quá', 'wá': 'quá', 'qá': 'quá',
    'đx': 'được', 'dk': 'được', 'dc': 'được', 'đk': 'được', 'đc': 'được',
    'vs': 'với', 'j': 'gì', '“': ' ', 'time': 'thời gian', 'm': 'mình', 'mik': 'mình', 'r': 'rồi', 'bjo': 'bao giờ', 'very': 'rất',

    'authentic': 'chuẩn chính hãng', 'aut': 'chuẩn chính hãng', 'auth': 'chuẩn chính hãng', 'date': 'hạn sử dụng', 'hsd': 'hạn sử dụng',
    'store': 'cửa hàng', 'sop': 'cửa hàng', 'shopE': 'cửa hàng', 'shop': 'cửa hàng',
    'sp': 'sản phẩm', 'product': 'sản phẩm', 'hàg': 'hàng',
    'ship': 'giao hàng', 'delivery': 'giao hàng', 'síp': 'giao hàng', 'order': 'đặt hàng',

    'gud': 'tốt', 'wel done': 'tốt', 'good': 'tốt', 'gút': 'tốt', 'tot': 'tốt', 'nice': 'tốt', 'perfect': 'rất tốt',
    'quality': 'chất lượng', 'chất lg': 'chất lượng', 'chat': 'chất', 'excelent': 'hoàn hảo', 'bt': 'bình thường',
    'sad': 'tệ', 'por': 'tệ', 'poor': 'tệ', 'bad': 'tệ',
    'beautiful': 'đẹp tuyệt vời', 'dep': 'đẹp',
    'xau': 'xấu', 'sấu': 'xấu',

    'thik': 'thích', 'iu': 'yêu', 'fake': 'giả mạo',
    'quickly': 'nhanh', 'quick': 'nhanh', 'fast': 'nhanh',
    'fresh': 'tươi', 'delicious': 'ngon',

    'dt': 'điện thoại', 'fb': 'facebook', 'face': 'facebook', 'ks': 'khách sạn', 'nv': 'nhân viên',
    'nt': 'nhắn tin', 'ib': 'nhắn tin', 'tl': 'trả lời', 'trl': 'trả lời', 'rep': 'trả lời',
    'fback': 'feedback', 'fedback': 'feedback',
    'sd': 'sử dụng', 'sài': 'xài',

    '^_^': 'tích cực', ':)': 'tích cực', ':(': 'tiêu cực',
    '❤️': 'tích cực', '👍': 'tích cực', '🎉': 'tích cực', '😀': 'tích cực', '😍': 'tích cực', '😂': 'tích cực', '🤗': 'tích cực', '😙': 'tích cực', '🙂': 'tích cực',
    '😔': 'tiêu cực', '😓': 'tiêu cực',
    '⭐': 'star', '*': 'star', '🌟': 'star',
}

with open('/content/drive/MyDrive/NLU_NCKH/notebook/Res_ABSA/teencode.txt', encoding='utf-8') as f:
    for pair in f.readlines():
        key, value = pair.split('\t')
        replace_list[key] = value.strip()


def normalize_acronyms(text):
    words = []
    for word in text.strip().split():
        # word = word.strip(string.punctuation)
        if word.lower() not in replace_list.keys(): words.append(word)
        else: words.append(replace_list[word.lower()])
    return emoji.demojize(' '.join(words)) # Remove Emojis

# Remove unnecessary characters
def remove_unnecessary_characters(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ_]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text

def preprocess_fn(text, word_tokenize = None):
    text = text.strip()
    #tokens = sent.split()
    #for token in tokens:
        #for t in token:
            #if t.isnumeric() or t.isdigit():
                #tokens.remove(token)
                #break
    text = remove_HTML(text)
    text = convert_unicode(text)
    text = standardize_sentence_typing(text)
    text = normalize_acronyms(text)

    tokens = text.split()
    if word_tokenize is None:
        text= underthesea.word_tokenize(" ".join(tokens), format="text")
    else:
        #for using VnCoreNLP
        text = " ".join(word_tokenize.tokenize(" ".join(tokens))[0])


    text = remove_unnecessary_characters(text)
    # return text.lower()
    return text

## Save data preprocesed

In [39]:
!pwd

/content/drive/MyDrive/NLU_NCKH/notebook


In [40]:
# Save data was preprocessed
train['review'] = train['review'].apply(preprocess_fn)
dev['review'] = dev['review'].apply(preprocess_fn)
test['review'] = test['review'].apply(preprocess_fn)

train.to_csv('./hotel_data_pre/Train.csv', index=False)
dev.to_csv('./hotel_data_pre/Dev.csv', index=False)
test.to_csv('./hotel_data_pre/Test.csv', index=False)

In [41]:
def mo2ml(y, columns):
    """Convert multi-output to multi-label data
    """
    sentiments = ['-', 'o', '+']
    newcols = [f'{a} {s}' for a in columns for s in sentiments]
    # print(newcols)
    nrows, ncols = len(y), len(newcols)

    ml = pd.DataFrame(np.zeros((nrows, ncols), dtype='bool'),
                      columns=newcols)

    for i, a in enumerate(columns):
        for j in range(1, 4):
            indices = y[a] == j
            # print(f'indices:\n{indices}\ty: {i * 3 + j - 1}\ni: {i}\ta: {a}\t')
            ml.iloc[indices, i * 3 + j - 1] = True

    return ml

## Multi-output to Multi-label

In [ ]:
ytrain_ml = mo2ml(ytrain, aspects)
ydev_ml   = mo2ml(ydev, aspects)
ytest_ml  = mo2ml(ytest, aspects)

In [ ]:
xtrain = Xtrain.apply(preprocess_fn)
xdev   = Xdev.apply(preprocess_fn)
xtest  = Xtest.apply(preprocess_fn)

# Statistic and Analysis

In [ ]:
def get_num_words_per_sample(sample_texts):
    """Gets the median number of words per sample given corpus.

    # Arguments
        sample_texts: list, sample texts.

    # Returns
        int, median number of words per sample.
    """
    num_words = [len(s.split()) for s in sample_texts]
    return np.median(num_words)

In [ ]:
num_samples = len(xtrain)
num_aspects = len(aspects)
num_classes = num_aspects * 3
num_words_per_sample = get_num_words_per_sample(xtrain)
sw_ratio = num_samples / num_words_per_sample


print("Xtrain key metrics")
print("Number of samples:", num_samples)
print("Number of aspects:", num_aspects)
print("Number of classes:", num_classes)
print("Number of words per sample:", num_words_per_sample)
print("Number of samples/number of words per sample ratio", sw_ratio)

Xtrain key metrics
Number of samples: 7028
Number of aspects: 12
Number of classes: 36
Number of words per sample: 13.0
Number of samples/number of words per sample ratio 540.6153846153846


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_pie(df, name):
    count = df.sum(axis=0)
    return go.Pie(labels=count.index, values=count,
                  textposition='inside', name=name)


names = ('Train', 'Dev', 'Test')
phaseA = (ytrain, ydev, ytest)

fig = make_subplots(cols=3, subplot_titles=names,
                    specs=[[{'type': 'pie'}] * 3])

for i, (df, name) in enumerate(zip(phaseA, names), 1):
    fig.add_trace(get_pie(df, name), row=1, col=i)

fig.update_layout(title='% of samples per aspect')

In [ ]:
phaseML = (ytrain_ml, ydev_ml, ytest_ml)

fig = make_subplots(cols=3, subplot_titles=names,
                    specs=[[{'type': 'pie'}] * 3])

for i, (df, name) in enumerate(zip(phaseML, names), 1):
    fig.add_trace(get_pie(df, name), row=1, col=i)

fig.update_layout(title='% of samples per class (entity, sentiment)')

In [ ]:
count0 = [len(s) for s in xtrain]
count1 = [len(s) for s in xdev]
count2 = [len(s) for s in xtest]

fig = go.Figure()
fig.add_trace(go.Histogram(x=count0, name='train'))
fig.add_trace(go.Histogram(x=count1, name='dev'))
fig.add_trace(go.Histogram(x=count2, name='test'))

fig.update_layout(title='Sample length distribution', barmode='overlay')
fig.update_traces(opacity=0.5)

In [ ]:
train_vocabulary = set()
for line in xtrain:
    words = line.strip().split()
    train_vocabulary.update(words)

# Tạo từ vựng của tập test
test_vocabulary = set()
for line in xtest:
    words = line.strip().split()
    test_vocabulary.update(words)

# Tạo từ vựng của tập val
val_vocabulary = set()
for line in xdev:
    words = line.strip().split()
    val_vocabulary.update(words)

In [ ]:
print("Số lượng từ vựng tập train: ", len(train_vocabulary))
print("Số lượng từ vựng tập val: ", len(val_vocabulary))
print("Số lượng từ vựng tập test: ", len(test_vocabulary))

print("Số lượng từ vựng nằm ngoài tập train của tập val: ", len(val_vocabulary - train_vocabulary))
print("Số lượng từ vựng nằm ngoài tập train của tập test: ", len(test_vocabulary - train_vocabulary))

Số lượng từ vựng tập train:  7629
Số lượng từ vựng tập val:  2191
Số lượng từ vựng tập test:  3712
Số lượng từ vựng nằm ngoài tập train của tập val:  481
Số lượng từ vựng nằm ngoài tập train của tập test:  1115


# Text to Vector

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 3),
                             min_df=2, max_df=0.9)

# x data using basic clean up class and basic features extrator
xtrain_tfidf = vectorizer.fit_transform(xtrain)
xdev_tfidf   = vectorizer.transform(xdev)
xtest_tfidf  = vectorizer.transform(xtest)

# Define function to evalute

In [ ]:
import os
import math
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

label_map= {'None': 0,
            'negative':1,
            'neutral':2,
            'positive':3}

replacements={0: 'None',
              1: 'negative',
              2: 'neutral',
              3: 'positive'}
target_names = list(map(str, replacements.values()))

def aspect_detection_eval(y_test, y_pred):
  """
  y_test: grouth_true test, DataFrame
  y_pred: grouth_true predict, DataFrame
  """
  categories= y_pred.columns
  y_test= y_test.fillna('not_exist').replace(label_map).values.tolist()
  y_pred= y_pred.fillna('not_exist').replace(label_map).values.tolist()

  aspect_test = []
  aspect_pred = []

  for row_test, row_pred in zip(y_test, y_pred):
      for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
          aspect_test.append(bool(col_test) * categories[index])
          aspect_pred.append(bool(col_pred) * categories[index])

  print("## Aspect Detection Evaluate ##")
  print(classification_report(aspect_test, aspect_pred, digits=4, zero_division=1))
  return classification_report(aspect_test, aspect_pred, digits=4, zero_division=1, output_dict=True)


def sentiment_classification_eval(y_test, y_pred):
  """
  y_test: grouth_true test, DataFrame
  y_pred: grouth_true predict, DataFrame
  """
  categories= y_pred.columns
  y_test= y_test.fillna('not_exist').replace(label_map).values.tolist()
  y_pred= y_pred.fillna('not_exist').replace(label_map).values.tolist()

  y_test_flat = np.array(y_test).flatten()
  y_pred_flat = np.array(y_pred).flatten()
  target_names = list(map(str, replacements.values()))

  print("## Sentiment Classification Evaluate ##")
  print(classification_report(y_test_flat, y_pred_flat, target_names=target_names, digits=4))

  return classification_report(y_test_flat, y_pred_flat, digits=4, output_dict=True)

def combination_eval(y_test, y_pred):
  """
  y_test: grouth_true test, DataFrame
  y_pred: grouth_true predict, DataFrame
  """
  categories= y_pred.columns
  y_test= y_test.fillna('not_exist').replace(label_map).values.tolist()
  y_pred= y_pred.fillna('not_exist').replace(label_map).values.tolist()

  aspect_polarity_test = []
  aspect_polarity_pred = []

  for row_test, row_pred in zip(y_test, y_pred):
      for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
          aspect_polarity_test.append(f'{categories[index]},{replacements[col_test]}')
          aspect_polarity_pred.append(f'{categories[index]},{replacements[col_pred]}')

  print("## Combination Evaluate (Aspect + Polarity dection) ##")
  print(classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1))
  return classification_report(aspect_polarity_test, aspect_polarity_pred, digits=4, zero_division=1, output_dict=True)

In [ ]:
from sklearn.multioutput import MultiOutputClassifier as MOC
from sklearn.metrics import f1_score, classification_report

categories= list(ytest.columns)
def mo2df(y):
    if isinstance(y, pd.DataFrame):
        return y
    return pd.DataFrame(y, columns=categories)

def quick_f1(y_true, y_pred):
    y_pred = mo2ml(mo2df(y_pred))
    return round(f1_score(y_true, y_pred, average='micro', zero_division=0), 4)

def evaluate(model, X, y, average='micro'):
    yb_true  = mo2ml(y)

    yb_pred  = mo2df(model.predict(X))
    yb_pred  = mo2ml(yb_pred)

    return classification_report(yb_true, yb_pred, zero_division=0)

# Training

In [ ]:
from sklearn.multioutput import MultiOutputClassifier as MOC
from sklearn.multioutput import ClassifierChain as MOC_CC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

# SVM
clf_moc = MOC(LinearSVC(random_state=5))
clf_moc_cc = MOC_CC(LinearSVC(random_state=5))

clf_moc.fit(xtrain_tfidf, ytrain)
clf_moc_cc.fit(xtrain_tfidf, ytrain)

ypred_svm_moc = clf_moc.predict(xtest_tfidf)
ypred_svm_moc_cc = clf_moc_cc.predict(xtest_tfidf)

# MLP
# mlp = MOC(MLPClassifier())
# mlp.fit(xtrain_tfidf, ytrain)
# ypred_mlp = clf0.predict(xtest_tfidf)

# Evaluate

## Aspect detection

In [ ]:
# ypred_mlp = mlp.predict(xtest_tfidf)
ypred_svm_moc = pd.DataFrame(ypred_svm_moc, columns= categories)
ypred_svm_moc_cc = pd.DataFrame(ypred_svm_moc_cc, columns= categories)
# ypred_mlp= pd.DataFrame(ypred_mlp, columns= categories)

print("## SVM Multi Ouput Classifier ##")
aspect_report_SVM_moc = aspect_detection_eval(ytest, ypred_svm_moc)

print("## SVM Classifier Chain##")
aspect_report_SVM_moc_cc = aspect_detection_eval(ytest, ypred_svm_moc_cc)

# print("## MLP ##")
# aspect_report_MLP = aspect_detection_eval(ytest, ypred_mlp)

## SVM Multi Ouput Classifier ##
## Aspect Detection Evaluate ##
                          precision    recall  f1-score   support

                             0.9486    0.9882    0.9680     20627
        AMBIENCE#GENERAL     0.9306    0.7093    0.8050       227
           DRINKS#PRICES     0.8000    0.0851    0.1538        47
          DRINKS#QUALITY     0.7959    0.5764    0.6686       203
    DRINKS#STYLE&OPTIONS     0.8500    0.3953    0.5397       129
             FOOD#PRICES     0.6471    0.1964    0.3014       112
            FOOD#QUALITY     0.8104    0.7022    0.7524       554
      FOOD#STYLE&OPTIONS     0.8301    0.5927    0.6916       437
        LOCATION#GENERAL     1.0000    0.6346    0.7765       104
      RESTAURANT#GENERAL     0.8319    0.3944    0.5351       251
RESTAURANT#MISCELLANEOUS     0.9677    0.4138    0.5797       145
       RESTAURANT#PRICES     0.8971    0.5214    0.6595       117
         SERVICE#GENERAL     0.9711    0.7756    0.8624       303

         

## Polarity dection

In [ ]:
print("## SVM MOC ##")
polarity_report_SVM_moc = sentiment_classification_eval(ytest, ypred_svm_moc)
print("## SVM MOC CC ##")
polarity_report_SVM_moc_cc = sentiment_classification_eval(ytest, ypred_svm_moc_cc)
# print("## MLP ##")
# polarity_report_MLP = sentiment_classification_eval(ytest, ypred_mlp)

## SVM MOC ##
## Sentiment Classification Evaluate ##
              precision    recall  f1-score   support

        None     0.9486    0.9882    0.9680     20627
    negative     0.7324    0.3586    0.4815       435
     neutral     0.6347    0.3444    0.4465       691
    positive     0.7458    0.5855    0.6560      1503

    accuracy                         0.9312     23256
   macro avg     0.7653    0.5692    0.6380     23256
weighted avg     0.9221    0.9312    0.9232     23256

## SVM MOC CC ##
## Sentiment Classification Evaluate ##
              precision    recall  f1-score   support

        None     0.9517    0.9844    0.9678     20627
    negative     0.6936    0.3747    0.4866       435
     neutral     0.6019    0.3632    0.4531       691
    positive     0.7182    0.6055    0.6570      1503

    accuracy                         0.9301     23256
   macro avg     0.7414    0.5820    0.6411     23256
weighted avg     0.9214    0.9301    0.9234     23256



## Aspect + Polarity dection

In [ ]:
print("## SVM MOC ##")
aspect_polarity_report_svm_moc = combination_eval(ytest, ypred_svm_moc)
print("## SVM MOC CC ##")
aspect_polarity_report_svm_moc_cc = combination_eval(ytest, ypred_svm_moc_cc)

# print("## MLP ##")
# aspect_polarity_report_mlp = combination_eval(ytest, ypred_mlp)

## SVM MOC ##
## Combination Evaluate (Aspect + Polarity dection) ##
                                   precision    recall  f1-score   support

            AMBIENCE#GENERAL,None     0.9626    0.9930    0.9776      1711
        AMBIENCE#GENERAL,negative     0.7273    0.4324    0.5424        37
         AMBIENCE#GENERAL,neutral     0.3333    0.1333    0.1905        30
        AMBIENCE#GENERAL,positive     0.8633    0.7500    0.8027       160
               DRINKS#PRICES,None     0.9778    0.9995    0.9885      1891
           DRINKS#PRICES,negative     1.0000    0.0000    0.0000         5
            DRINKS#PRICES,neutral     0.8000    0.1250    0.2162        32
           DRINKS#PRICES,positive     1.0000    0.0000    0.0000        10
              DRINKS#QUALITY,None     0.9520    0.9827    0.9671      1735
          DRINKS#QUALITY,negative     0.5000    0.0385    0.0714        26
           DRINKS#QUALITY,neutral     0.6154    0.1667    0.2623        48
          DRINKS#QUALITY,posit

# Summary

In [ ]:
## SVM
aspect_dict_svm = aspect_report_SVM['macro avg']
aspect_dict_svm['accuracy'] = aspect_report_SVM['accuracy']

polarity_dict_svm  = polarity_report_SVM['macro avg']
polarity_dict_svm['accuracy'] = polarity_report_SVM['accuracy']

aspect_polarity_dict_svm = aspect_polarity_report_svm['macro avg']
aspect_polarity_dict_svm['accuracy'] = aspect_polarity_report_svm['accuracy']

## MLP
aspect_dict_mlp = aspect_report_MLP['macro avg']
aspect_dict_mlp['accuracy'] = aspect_report_MLP['accuracy']

polarity_dict_mlp  = polarity_report_MLP['macro avg']
polarity_dict_mlp['accuracy'] = polarity_report_MLP['accuracy']

aspect_polarity_dict_mlp = aspect_polarity_report_mlp['macro avg']
aspect_polarity_dict_mlp['accuracy'] = aspect_polarity_report_mlp['accuracy']

# Create dataframe for dict_score
df_report_svm = pd.DataFrame.from_dict([aspect_dict_svm, polarity_dict_svm, aspect_polarity_dict_svm])
df_report_svm.index = ['Aspect Detection', 'Polarity Detection', 'Aspect + Polarity']
df_report_svm = df_report_svm.drop('support', axis=1)

df_report_mlp = pd.DataFrame.from_dict([aspect_dict_mlp, polarity_dict_mlp, aspect_polarity_dict_mlp])
df_report_mlp.index = ['Aspect Detection', 'Polarity Detection', 'Aspect + Polarity']
df_report_mlp = df_report_mlp.drop('support', axis=1)

In [ ]:
df_report_svm

In [ ]:
df_report_mlp